In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [9]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (355.84 seconds): Collecting URLs... indiatimes          

n_total     26077
n_unique    26005
dtype: int64

,pubs,actual_pub
https://breitbart.com/2020-election/2019/10/15/cnn-new-york-times-moderators-fail-to-ask-about-china-hong-kong-democrat-debate,"realclearpolitics, breitbart",breitbart
https://businessinsider.co.za/cape-town-avoids-stage-2-load-shedding-eskom-city-of-cape-town-steenbras-dam-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/load-shedding-traffic-cape-town-johannesburg-metro-police-eskom-city-of-cape-town-road-intersections-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/photo-video-nasas-hubble-space-telescope-spots-interstellar-comet-2019-10,"businessinsider, news24",businessinsider
https://foxnews.com/opinion/david-bossie-7-democratic-debate-takeaways-after-watching-candidates-caught-up-in-trump-derangement-syndrome,"fox, realclearpolitics",fox
https://hotair.com/archives/john-s-2/2019/10/16/will-netflix-next-big-company-show-off-woke-hypocrisy,"townhall, hotair",hotair
https://msnbc.com/hardball/watch/schiff-gives-an-update-on-the-house-impeachment-investigation-71333957901,"msnbc, nbcnews",msnbc
https://msnbc.com/morning-joe/watch/elizabeth-warren-takes-heat-from-candidates-at-ohio-debate-71356997792,"msnbc, nbcnews",msnbc
https://msnbc.com/msnbc/watch/elizabeth-warren-and-joe-biden-clash-over-credit-for-one-of-her-signature-accomplishments-71352389539,"msnbc, nbcnews",msnbc
https://msnbc.com/msnbc/watch/rudy-giuliani-hand-grenade-under-investigation-71356485950,"msnbc, nbcnews",msnbc


huffpost                  78
cnn                      850
investing.com           1025
politico                 213
time                      25
cnbc                     173
fox                      323
bbc                      275
businessinsider          562
morningstar               96
wsj                      109
nytimes                  105
guardian                 163
reuters                 1147
washingtontimes          463
washingtonpost           131
cbs                      410
marketwatch              183
atlantic                 101
vice                     103
npr                      880
newrepublic               21
yahoo                    393
independent              719
heritage                 280
zdnet                    333
townhall                 627
abcnews                   97
hotair                    84
cbc                      299
                        ... 
metro                    242
msnbc                    114
nationalreview           737
news24        

In [10]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [11]:
select_results = crawler.select(); display(*select_results)

100.00% (36.31 seconds): Selecting URLs... cryptonews          

n_total     4203
n_unique    4203
dtype: int64

,pubs,actual_pub


huffpost                 11
cnn                      78
investing.com           232
politico                 22
time                      8
cnbc                     42
fox                     126
bbc                      42
businessinsider         190
morningstar              10
wsj                      34
nytimes                  29
guardian                 51
reuters                 393
washingtontimes          77
washingtonpost           64
cbs                      37
marketwatch              63
atlantic                 12
vice                     36
npr                      11
newrepublic               4
yahoo                   235
independent              48
heritage                  2
zdnet                    23
townhall                 41
abcnews                  35
hotair                   31
cbc                      58
                       ... 
metro                    92
msnbc                    24
nationalreview           53
news24                  124
techcrunch          

In [12]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [13]:
download_results = crawler.download(); download_results

21.77% (801.72 seconds): Downloading... politico            

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100.00% (4481.05 seconds): Downloading... dailymail           

,downloaded,trashed
abcnews,35,0
afr,8,0
aljazeera,24,83
americanconservative,11,2
arirang,0,0
arstechnica,9,0
arynews,12,28
atlantic,12,0
axios,42,9
bbc,40,2


In [14]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

['4446cd165d53df36fd33b7a80c7e34e90790aabf',
 '07813a79f1ed9846257dcdbf27fd8ad941cf8a26',
 '26e5abcc0b8068b3bb806099d39982458c661dad',
 'e2e16e8f236e4cf7c45598193ab68655f807e0dd']